##### The idea of the project was to create a virtual recruiter
<a href="https://docs.google.com/presentation/d/19Zd_xgWsEWjW9fEIpS615pnSRtO5o8hp_vvGZCQNI6Q/edit?usp=sharing"> link of virtual recruiter</a><br>

The is just a small use case out of that to generate a personalized email using candidates Name, Skills, Experience information we have 

###### Importing required packages
Please install the required packages

In [1]:
# Packages Lang Chain
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# Packages wrt Transformers 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration

2023-06-09 16:56:46.009730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Model name and hugging face urls wrt Flan
<ul>
    <li><a href="https://huggingface.co/google/flan-t5-base">flan-t5-base<a> 1GB+<br></li>
    <li><a href="https://huggingface.co/google/flan-t5-large">flan-t5-large<a> 3.5GB+<br></li>
    <li><a href="https://huggingface.co/google/flan-t5-xl">flan-t5-xl<a> 11GB+ <br></li>
</ul>    
As <b>Flan-t5-xl</b> is too large and takes <b>6-7+ hours</b> of run-time couldn't proceed, as we did't get a chance to <b>test the best prompts that suits our use-case</b>
    
##### Tried LLAMA
<a href="https://huggingface.co/decapoda-research/llama-7b-hf">llama's smallest model<a> 13GB+<br>
    
Same use issue as flan run-time is 10+ hours. Tried <b>Colab GPU</b> as well but <b>Available CPU RAM</b> is only <b>12 GB</b> it's breaking before loading the model.

<b>Colab</b>: Fails when you give <b>Candidate Name</b> even wrt large model in <b>free version</b> though you have <b>16 GB GPU</b> as of <b>June 2023</b>

In [2]:
model_id = 'google/flan-t5-large'

###### Loading the model

In [3]:
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)


#### This is which can be used without loading LLM template

In [4]:
def get_output(text, model=model, tokenizer=tokenizer, max_length=512, beams=32, penalty=2, ngram=2):
    """
    https://huggingface.co/google/flan-t5-base
    further fine-tune the params as per the requirement
    you can check more parameters from model.generate documentation and further fine-tune
    """
    input_ids = tokenizer(text, return_tensors="pt", max_length=max_length).input_ids
    outputs = model.generate(input_ids, max_new_tokens=max_length, length_penalty=penalty, num_beams=beams, no_repeat_ngram_size=ngram)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return text_output

##### Output without using Lanchain

<ul>
    <li>Please try various <b>templates</b> as they vary from model to model</li>
    <li><b>Template</b>: Way you give input to the model</li>
    <li><b>Colab</b>: Fails when you give <b>Candidate name</b> even though we have <b>GPU</b> available wrt below example</li>
</ul>

In [5]:
%%time
John_text = "Context about John:\n 5+ years of experience as a Sr. Financial Analyst.\n \
Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
monthly and quarterly reporting, good oral and written skills.\n\n\n Write a personalized email \
from recruiter to John saying he is good fit for our company?"
get_output(John_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 4h 31min 10s, sys: 2h 34min 34s, total: 7h 5min 45s
Wall time: 1h 26min 52s


'Dear John, Thank you for your interest in our company. We are currently looking for a Sr. Financial Analyst to join our team. John has 5+ years of experience in the financial industry. He is well versed in business intelligence, retention forecasts, statistical analysis, presentation skills, monthly and quarterly reporting, good oral and written skills. Please let me know if you have any questions or would like to schedule an interview. I look forward to speaking with you soon. Thanks again and we wish you the best of luck in your future endeavors. -Join our email list to receive job alerts and job opportunities. You can unsubscribe at any time by replying to this email with the word \'UNSUBSCRIBE\' at the bottom of the email. (Please include your name, phone number, city, state, and zip code). You will receive an email every time new job matching your skills and interests are added to the list. Your email will not be shared with anyone else and will only be used for the sole purpose 

In [6]:
%%time
John_text_1 = "Candidate Name: John\n Experience: 5 years\n Job Title: Sr. Financial Analyst\n \
Skills: Business Intelligence, Retention forecasts, Statistical Analysis, Monthly and Quarterly reporting, \
good oral and written skills.\n\n Write a personalized email from recruiter to John \
saying he is good fit for our company?"
get_output(John_text_1)

CPU times: user 5h 33min 6s, sys: 2h 46min 12s, total: 8h 19min 19s
Wall time: 1h 9min 1s


"Thank you for your interest in the position of Sr. Financial Analyst. We are currently accepting applications for this position. If you are interested in learning more about the role or would like to apply, please respond to this email with your cover letter, resume, and contact information. I look forward to speaking with you soon. Please forward this message to anyone you think would be a good fit for our company. You can also call me at the number listed below, or send me an email using the form on the right side of this page. Let me know if you have any questions and I will be happy to assist you in any way I can. In the meantime, I wish you the best of luck in your endeavors and hope to hear from you as soon as possible. Thanks again for taking the time to review this job advertisement and please don't hesitate to contact me with any additional questions or concerns you may have. Your response is very important to us and we will do everything we can to ensure that you get the mos

##### Output wrt other names with same information 
Flan is failing badly as it's a probability based model

In [7]:
%%time
Ganesh_text = "Context about Ganesh:\n 5+ years of experience as a Sr. Financial Analyst.\n \
Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
monthly and quarterly reporting, good oral and written skills.\n\n\n Write a personalized email \
from recruiter to Ganesh saying he is good fit for our company?"
get_output(Ganesh_text)

CPU times: user 34min 7s, sys: 6min 15s, total: 40min 22s
Wall time: 4min 31s


'Ganesh has 5+ years of experience as a Sr. Financial Analyst. Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, monthly and quarterly reporting, good oral and written skills. He is good fit for our company.'

In [8]:
%%time
Ganesh_text_1 = "Candidate Name: Ganesh\n Experience: 5 years\n Job Title: Sr. Financial Analyst\n \
Skills: Business Intelligence, Retention forecasts, Statistical Analysis, Monthly and Quarterly reporting, \
good oral and written skills.\n\n Write a personalized email from recruiter to Ganesh \
saying he is good fit for our company?"
get_output(Ganesh_text_1)

CPU times: user 1h 4min 51s, sys: 14min 13s, total: 1h 19min 5s
Wall time: 8min 47s


"Thank you for your interest in our company. We are currently looking for a Sr. Financial Analyst to join our team. Ganesh has 5 years of experience in the financial industry. He is well versed in Business Intelligence, Retention forecasts, Statistical Analysis, Monthly and Quarterly reporting, good oral and written skills. Please let me know if you are interested in this position and I will forward your resume to you as soon as possible. I look forward to hearing from you. If you have any questions, please don't hesitate to contact me at the number listed below. Best regards,"

#### Loading the model using LLM and template

In [9]:
def load_llm(model_id, model=model, tokenizer=tokenizer, max_length=512):
    """
    loading model and tokenizer
    """
    try:
        if model is None:
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
        pipe = pipeline(
            "text2text-generation",
            model=model, 
            tokenizer=tokenizer,
            max_length=max_length
        )
        local_llm = HuggingFacePipeline(pipeline=pipe)
        return local_llm
    except Exception as ex:
        print(ex)
    return None

##### Loading the LLM

In [10]:
local_llm = load_llm(model_id)
local_llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7fd566c68520>, model_id='gpt2', model_kwargs=None)

###### If you wanna use template please fine tune it as per your requirement
This template can be re-used for all the models

In [11]:
class CustomTemplate:
    """
    generic template
    """
    def __init__(self,llm, prompt):
        """
        creating template by defaul once
        """
        self.llm = llm
        self.llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    def get_output(self, data):
        """
        running on the required data
        """
        try:
            output = self.llm_chain.run(data)
            return output
        except Exception as ex:
            print(ex)
        return None
    
    def get_output_new_template(self, custom_prompt, variables, text):
        try:
            prompt = PromptTemplate(template=custom_prompt,input_variables=variables)
            chain_temp = LLMChain(prompt=prompt, llm=self.llm)
            output = chain_temp.run(text)
            chain_temp = None
            return output
        except Exception as ex:
            print(ex)
        return None
    
    def get_question_answer(self,question):
        """
        just question answering output
        """
        try:
            output = self.llm(question)
            return output
        except Exception as ex:
            print(ex)
        return None
        
    

##### Defining the template to test

In [12]:
test_template = """
     Question: {question}

    Answer: Let's think step by step."""
test_prompt = PromptTemplate(template=test_template, input_variables=["question"])

##### Testing the model

In [13]:
test_model = CustomTemplate(local_llm, test_prompt)

##### Checking question answer output

In [14]:
test_model.get_question_answer("What is the capital of France?")

'paris'

#### Testing output wrt Template

In [15]:
test_model.get_output({"question":"What is the capital of England?"})

'The capital of England is London. London is the capital of England. So the answer is London.'

##### Clearing the memory

In [16]:
test_model=None

#### Fine-tuned llm

In [17]:
def load_llm(model_id, model=model, tokenizer=tokenizer, max_length=512, beams=32, penalty=2, ngram=2):
    """
    loading model and tokenizer
    please feel free to fine-tune parameters as per the requirement
    num_beams is beam search
    for all the parameters to fine-tune in detail explore model.generate documentation
    """
    try:
        if model is None:
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
        pipe = pipeline(
            "text2text-generation",
            model=model, 
            tokenizer=tokenizer, 
            max_length=max_length,
            length_penalty=penalty, 
            num_beams=beams, 
            no_repeat_ngram_size=ngram,
        )
        local_llm = HuggingFacePipeline(pipeline=pipe)
        return local_llm
    except Exception as ex:
        print(ex)
    return None

#### Loading fine-tuned LLm

In [18]:
local_llm = load_llm(model_id)
local_llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7fd326fa2c50>, model_id='gpt2', model_kwargs=None)

##### Template wrt text2text

Please experiment with custom templates as per requirements which is default model can be accessed using get_output

In [19]:
prompt = PromptTemplate(
            input_variables=["instruction", "context"],
            template="""Context: ```{context}```\n\n            
            Question: {instruction}\n\n            
            Answer: """)

###### Loading the model

In [20]:
text2text = CustomTemplate(local_llm, prompt)

##### Testing

Don't run this due to beam search it would take lot of time

In [17]:
%%time
#text2text.get_question_answer("What is the capital of USA?")

CPU times: user 2h 13min 15s, sys: 59min 9s, total: 3h 12min 25s
Wall time: 23min 40s


'san francisco california united states of america -lrb- u.s. nasdaq : ussr ; also known as los angeles and new york city ) is the capital city of the United States of America and the largest city in the continental usa with a population of over 7 million people residing in metropolitan denver , colorado and rio grande as well as tens of thousands of small towns and unincorporated communities throughout the contiguous states and in gulf of mexico. it is also the birthplace of many cultures and religions including judaism, christianity and bohemia and jerusalem whose names are derived from greek words meaning ‘beautiful’ or ‘pretty’ and ‘semi-rural’ – both of which have been used by indigenous peoples for eons to describe their cosmopolitan culture and people to which they have referred to as ‘americans’ to distinguish them from non-indigenous people and to help them understand and appreciate the uniqueness of their own idiosyncrasies and customs which are unique to each and every one o

##### Experimenting using the above prompt template
Please try to add <b>answer format</b> as well which would make you job much easier which I have missed it. As project is halted haven't explored that

In [21]:
data = {"instruction" : "Write a personalized email for John from recruiter?",
        "context" : "John has 5+ years of experience as a Sr. Financial Analyst. \
        Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
        monthly and quarterly reporting, good oral and written skills."}

##### Testing on Sample Data

In [22]:
%%time
text2text.get_output(data)

CPU times: user 4h 48min 47s, sys: 2h 13min 18s, total: 7h 2min 5s
Wall time: 54min 20s


"Dear John, Thank you for your interest in our company. We are currently looking for a Sr. Financial Analyst to join our team. If you are interested in learning more about this position, please respond to this email with your cover letter, resume, and contact information. I look forward to speaking with you soon. Thanks again for considering us in your search for the right person. Recruiter: Enron North America, L.L.C. Job #: C/S/Financial_Analyst/Ph.D. Location: Houston, Texas, U.S.A. Job Category: Analyst / Manager Job Tags: Business Intelligence, Retention Forecasts, Statistical Analysis, Presentation Skills, Monthly and Quarterly Reporting, Good Oral and Written Skills. JOB SUMMARY: The ideal candidate will have 5+ years of experience in the financial industry. He or she will be responsible for business intelligence, retention forecasts and statistical analysis, presentation skills, monthly and quarterly reporting, good oral and written skills. This position will report directly to

In [23]:
%%time
data1 = {"instruction" : "Write a personalized email to John from the recruiter requesting him to \
consider the opportinity as Sr.Financial Analyst job in our company?",
         "context" : "John has 5+ years of experience as a Sr. Financial Analyst. \
         Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
         monthly and quarterly reporting, good oral and written skills."
        }

text2text.get_output(data1)

CPU times: user 58min 4s, sys: 12min 35s, total: 1h 10min 40s
Wall time: 7min 57s


'Dear John, I would like to invite you for an interview for the position of Sr.Financial Analyst in our company. We are looking for a person with 5+ years of experience in the field of financial analysis. If you are interested in this position, please forward your resume to me and I will get back to you as soon as possible. I look forward to hearing from you. Thanks for your time and please let me know if there is anything else I can do to help you in your job search. Regards,'

#### Testing on Custom Template
Templates vary from model to Model freel free to explore as per the model

In [24]:
temp1 = """
Write a personalized email to John from the recruiter \
requesting him to consider the opportinity as Sr.Financial Analyst job in our company?

{text}

"""

#### Running on custom template

In [25]:
%%time
john_text = "John has 5+ years of experience as a Sr. Financial Analyst. \
Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
monthly and quarterly reporting, good oral and written skills."
text2text.get_output_new_template(temp1, ["text"], john_text)

CPU times: user 5h 26min 50s, sys: 2h 35min 41s, total: 8h 2min 32s
Wall time: 1h 3min 24s


'John has 5+ years of experience as a Sr. Financial Analyst. Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, monthly and quarterly reporting, good oral and written skills. The recruiter requesting him to consider the opportinity as S.Financial Analyst job in our company. If you are interested in this position, please send your resume to the following e-mail address: recruiter@employeecareers.com. Please reply to this email with the subject line "JOB DESCRIPTION" and your name and phone number so that we can get in touch with you as soon as possible. Thank you for your time and I look forward to hearing from you. You can also call me at the number given below to speak with one of our recruiters if you have any questions regarding this job or would like to know more about this opportunity. Thanks again for taking the time to read this message. I wish you the best of luck in your future endeavors and hope that you find the position that be

###### Custom Template 2 
Add answer as well and output format wrt answer

In [26]:
temp2 = """
Your task is to generate a descriptive email to John requesting him to \
join our company. Start by saying how impressed we are with his experience \
and skills. Share information about why he is a great fit in our company, \
what we offer from the learning curve, and as a team, benefits.  
  

Candidate Information: {text}
"""


In [27]:
%%time
john_text = "John has 5+ years of experience as a Sr. Financial Analyst. \
Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, \
monthly and quarterly reporting, good oral and written skills."
text2text.get_output_new_template(temp2, ["text"], john_text)

CPU times: user 46min 20s, sys: 9min 56s, total: 56min 17s
Wall time: 6min 17s


'John has 5+ years of experience as a Sr. Financial Analyst. Skills are business intelligence, retention forecasts, statistical analysis, presentation skills, monthly and quarterly reporting, good oral and written skills. I would like to invite John to join our team. Please respond to this email and let me know if you have any questions. Thank you for your time and I look forward to hearing from you.'

##### Flan-t5 vs Chat-gpt-3.5
<a href="https://docs.google.com/document/d/1HaGaBwtTS06VI56Q5397zxWVSdNrvqyQMKi22Y3qRVk/edit?usp=sharing"> results link </a>

Please feel to futher add <b>Answer</b> format as well and experiement.
Please do the following:
<ul>
    <li>Latest models</li>
    <li>Fine-tune Model Parameters</li>
    <li>Scalability(Throughtput and Inference)</li>
    <li>LLM params</li>
    <li>Prompts which very important varies model to model</li>
</ul>